## Imports

In [110]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [111]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=2)

# Get the date one week ago
one_week_ago = today - timedelta(days=8)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

May. 17th May. 25th


In [112]:
filename = glob.glob('*redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [113]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [114]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [115]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [116]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,May-20-2025,Condo/Co-op,10101 Collins Ave Unit 15A,Bal Harbour,FL,33154.0,3450000.0,3.0,4.5,THE PALACE AT BAL HARBOUR,3110.0,NaN,1994.0,NaN,1109.0,4755.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10101-Co...,MARMLS,A11799132,N,Y,25.892679,-80.122902


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [117]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [118]:
df2 = df[(df['SOLD DATE'] > one_week_ago) & (df['SOLD DATE'] < today)]

In [119]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-05-23    26
2025-05-20    22
2025-05-22    19
2025-05-21    19
Name: count, dtype: int64

In [120]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [121]:
df_top_ten = df2.head(10)

In [122]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
63,PAST SALE,2025-05-20,Condo/Co-op,1425 Brickell Ave Ph 2abcd,Miami,FL,33131.0,14800000.0,4.0,4.0,Four Seasons Residences,7438.0,NaN,2003.0,NaN,1990.0,9800.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1425-Brickell-...,MARMLS,A11702498,N,Y,25.759088,-80.191817
69,PAST SALE,2025-05-22,Condo/Co-op,9349 Collins Ave #1004,Surfside,FL,33154.0,9000000.0,3.0,4.5,ALTOS DEL MAR NO. 5,3334.0,NaN,2016.0,NaN,2699.0,9757.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9349-Collin...,MARMLS,A11693223,N,Y,25.883056,-80.121818
62,PAST SALE,2025-05-21,Condo/Co-op,300 Biscayne Blvd Way Unit 4802 Duplex,Miami,FL,33131.0,7750000.0,5.0,6.0,Aston Martin Residences,4434.0,NaN,2024.0,NaN,1748.0,7019.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/300-Biscayne-B...,MARMLS,A11691203,N,Y,25.770777,-80.187521
57,PAST SALE,2025-05-23,Condo/Co-op,300 Biscayne Blvd Way #601,Miami,FL,33131.0,3600000.0,4.0,4.5,ASTON MARTIN CONDO,3360.0,NaN,2024.0,NaN,1071.0,5720.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/300-Biscayne-B...,MARMLS,A11728779,N,Y,25.770777,-80.187521
31,PAST SALE,2025-05-21,Condo/Co-op,2627 S Bayshore Dr #1201,Miami,FL,33133.0,3600000.0,3.0,3.0,GROVENOR HOUSE,2333.0,NaN,2005.0,NaN,1543.0,3606.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,MARMLS,A11748546,N,Y,25.732272,-80.234921
1,PAST SALE,2025-05-20,Condo/Co-op,10101 Collins Ave Unit 15A,Bal Harbour,FL,33154.0,3450000.0,3.0,4.5,THE PALACE AT BAL HARBOUR,3110.0,NaN,1994.0,NaN,1109.0,4755.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10101-Co...,MARMLS,A11799132,N,Y,25.892679,-80.122902
9,PAST SALE,2025-05-23,Condo/Co-op,1643 Brickell Ave #1201,Miami,FL,33129.0,2900000.0,3.0,4.5,SANTA MARIA CONDO,3170.0,NaN,1997.0,NaN,915.0,3626.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1643-Brickell-...,MARMLS,A11659208,N,Y,25.755513,-80.195018
45,PAST SALE,2025-05-21,Condo/Co-op,488 NE 18th St #4312,Miami,FL,33132.0,2350000.0,3.0,4.0,ARIA ON THE BAY CONDO,2018.0,NaN,2018.0,NaN,1165.0,2239.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/488-NE-18th-St...,MARMLS,A11736645,N,Y,25.793659,-80.186979
86,PAST SALE,2025-05-23,Condo/Co-op,4775 Collins Ave #1602,Miami Beach,FL,33140.0,2175000.0,2.0,2.0,GREEN DIAMOND CONDO,1530.0,NaN,2000.0,NaN,1422.0,2062.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/4775-Col...,MARMLS,A11504621,N,Y,25.822419,-80.121616
3,PAST SALE,2025-05-23,Condo/Co-op,151 Crandon Blvd #604,Key Biscayne,FL,33149.0,1950000.0,3.0,3.0,THE EMERALDBAY @ KEY COLO,2010.0,NaN,1980.0,NaN,970.0,2526.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/151-Cra...,MARMLS,A11802092,N,Y,25.696473,-80.161501


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [123]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [124]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [125]:
response_df = pd.DataFrame(response_list)

In [126]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [127]:
df_top_ten = merged_df

In [128]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2
0,PAST SALE,2025-05-20,Condo/Co-op,1425 Brickell Ave Ph 2abcd,Miami,FL,33131.0,14800000.0,4.0,4.0,Four Seasons Residences,7438.0,NaN,2003.0,NaN,1990.0,9800.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1425-Brickell-...,MARMLS,A11702498,N,Y,25.759088,-80.191817,https://www.redfin.com/FL/Miami/1425-Brickell-...,Gabriela Dajer,One Sotheby's International Realty,None,None,Alejandra Anzola,Fortune International Realty,None,None
1,PAST SALE,2025-05-22,Condo/Co-op,9349 Collins Ave #1004,Surfside,FL,33154.0,9000000.0,3.0,4.5,ALTOS DEL MAR NO. 5,3334.0,NaN,2016.0,NaN,2699.0,9757.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9349-Collin...,MARMLS,A11693223,N,Y,25.883056,-80.121818,https://www.redfin.com/FL/Surfside/9349-Collin...,None,None,None,None,None,None,None,None
2,PAST SALE,2025-05-21,Condo/Co-op,300 Biscayne Blvd Way Unit 4802 Duplex,Miami,FL,33131.0,7750000.0,5.0,6.0,Aston Martin Residences,4434.0,NaN,2024.0,NaN,1748.0,7019.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/300-Biscayne-B...,MARMLS,A11691203,N,Y,25.770777,-80.187521,https://www.redfin.com/FL/Miami/300-Biscayne-B...,Silvia Coltrane,The Corcoran Group,None,None,Ivan Chorney,"Compass Florida, LLC.",None,None
3,PAST SALE,2025-05-23,Condo/Co-op,300 Biscayne Blvd Way #601,Miami,FL,33131.0,3600000.0,4.0,4.5,ASTON MARTIN CONDO,3360.0,NaN,2024.0,NaN,1071.0,5720.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/300-Biscayne-B...,MARMLS,A11728779,N,Y,25.770777,-80.187521,https://www.redfin.com/FL/Miami/300-Biscayne-B...,Marko Gojanovic,One Sotheby's International Re,Reid Heidenry,One Sotheby's International Re,Maria Sendrowski,Xcellence Realty,None,None
4,PAST SALE,2025-05-21,Condo/Co-op,2627 S Bayshore Dr #1201,Miami,FL,33133.0,3600000.0,3.0,3.0,GROVENOR HOUSE,2333.0,NaN,2005.0,NaN,1543.0,3606.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,MARMLS,A11748546,N,Y,25.732272,-80.234921,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,Maria Cerna,EXP Realty LLC,None,None,Ana Sanchez,One Sotheby's International Realty,Yedri Villafane,One Sotheby's International Realty
5,PAST SALE,2025-05-20,Condo/Co-op,10101 Collins Ave Unit 15A,Bal Harbour,FL,33154.0,3450000.0,3.0,4.5,THE PALACE AT BAL HARBOUR,3110.0,NaN,1994.0,NaN,1109.0,4755.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10101-Co...,MARMLS,A11799132,N,Y,25.892679,-80.122902,https://www.redfin.com/FL/Bal-Harbour/10101-Co...,Raquel Szyller,"Aventura Real Estate, Inc.",None,None,Nicole Bouhadana,Beachfront Realty Inc,None,None
6,PAST SALE,2025-05-23,Condo/Co-op,1643 Brickell Ave #1201,Miami,FL,33129.0,2900000.0,3.0,4.5,SANTA MARIA CONDO,3170.0,NaN,1997.0,NaN,915.0,3626.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1643-Brickell-...,MARMLS,A11659208,N,Y,25.755513,-80.195018,https://www.redfin.com/FL/Miami/1643-Brickell-...,Ingrid Prieto de Ducournau,New Miami Realty,None,None,Ingrid Prieto de Ducournau,New Miami Realty,None,None
7,PAST SALE,2025-05-21,Condo/Co-op,488 NE 18th St #4312,Miami,FL,33132.0,2350000.0,3.0,4.0,ARIA ON THE BAY CONDO,2018.0,NaN,2018.0,NaN,1165.0,2239.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/488-NE-18th-St...,MARMLS,A11736645,N,Y,25.793659,-80.186979,https://www.redfin.com/FL/Miami/488-NE-18th-St...,Alessandro Saponaro,AFS Living,None,None,Goritza Draskovich,Fortune International Realty,None,None
8,PAST SALE,2025-05-23,Condo/Co-op,4775 Collins Ave #1602,Miami Beach,FL,33140.0,2175000.0,2.0,2.0,GREEN DIAMOND CONDO,1530.0,NaN,2000.0,NaN,1422.0,2062.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/4775-Col...,MARMLS,A11504621,N,Y,25.822419,

## Current Week's Values

In [129]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

86


In [130]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$1,023,844


In [131]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$542


In [132]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$88,050,620


In [133]:
def load_last_week_data_df(filename='last_week_data.csv'):
    try:
        return pd.read_csv(filename)
    except FileNotFoundError:
        return None
    
last_week = load_last_week_data_df()

In [134]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
# previous_week_condo_sales = input()
previous_week_condo_sales = last_week['Condo Sales'].iloc[0]
print(previous_week_condo_sales)
print('-----------')

print('Input Previous Week Condo Average Sales Price:')
# previous_week_average_sales_price = input()
previous_week_average_sales_price = last_week['Average Sale Price'].iloc[0]
print(previous_week_average_sales_price)
print('-----------')

print('Input Previous Week Condo Average PSF:')
# previous_week_average_psf = input()
previous_week_average_psf = last_week['Average PSF'].iloc[0]
print(previous_week_average_psf)
print('-----------')

print('Input Previous Week Condo Sales total (ex: 198_000_000)')
# previous_week_condo_sales_total = input()
previous_week_condo_sales_total = last_week['Last Week Total Dollar Volume'].iloc[0]
print(previous_week_condo_sales_total)


## Clean Data

Input county name:
Miami-Dade
Input Previous Week Condo Sales Total (number units sold):
127
-----------
Input Previous Week Condo Average Sales Price:
973842
-----------
Input Previous Week Condo Average PSF:
636
-----------
Input Previous Week Condo Sales total (ex: 198_000_000)
123677931.0


In [135]:
last_week

,Condo Sales,Average Sale Price,Average PSF,Last Week Total Dollar Volume
0,127,973842,636,123677931.0


In [136]:
last_week['Last Week Total Dollar Volume'].iloc[0]

123677931.0

In [137]:
def save_current_week_data_df(df, filename='last_week_data.csv'):
    df.to_csv(filename, index=False)

# Create a dictionary of your current week data.
current_week_data = {
    "Condo Sales": current_week_condo_sales,
    "Average Sale Price": average_sale_price,
    "Average PSF": average_price_psf,
    "Last Week Total Dollar Volume": current_week_condo_sales_total
}

# Convert the dictionary into a DataFrame.
current_week_summary_df = pd.DataFrame([current_week_data])

# Save the DataFrame to a CSV file.
save_current_week_data_df(current_week_summary_df)

# Next week, load last week's data automatically.
last_week_df = load_last_week_data_df()
if last_week_df is not None:
    print("Last week's data:")
    print(last_week_df)
else:
    print("No previous data found.")

Last week's data:
   Condo Sales  Average Sale Price  Average PSF  Last Week Total Dollar Volume
0           86             1023844          542                     88050620.0


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [138]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [139]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [140]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [141]:
pd.set_option('display.max_columns',None)

In [142]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-05-20,Condo/Co-op,1425 Brickell Ave Ph 2abcd,Miami,FL,33131.0,14800000.0,4.0,4.0,Four Seasons Residences,7438.0,NaN,2003.0,NaN,1990.0,9800.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1425-Brickell-...,MARMLS,A11702498,N,Y,25.759088,-80.191817,https://www.redfin.com/FL/Miami/1425-Brickell-...,Gabriela Dajer,One Sotheby's International Realty,None,None,Alejandra Anzola,Fortune International Realty,None,None,1,orange


In [143]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [144]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-05-20,Condo/Co-op,1425 Brickell Ave Ph 2abcd,Miami,FL,33131.0,14800000.0,4.0,4.0,Four Seasons Residences,7438.0,NaN,2003.0,NaN,1990.0,9800.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1425-Brickell-...,MARMLS,A11702498,N,Y,25.759088,-80.191817,https://www.redfin.com/FL/Miami/1425-Brickell-...,Gabriela Dajer,One Sotheby's International Realty,None,None,Alejandra Anzola,Fortune International Realty,None,None,1,orange
1,PAST SALE,2025-05-22,Condo/Co-op,9349 Collins Ave #1004,Surfside,FL,33154.0,9000000.0,3.0,4.5,ALTOS DEL MAR NO. 5,3334.0,NaN,2016.0,NaN,2699.0,9757.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9349-Collin...,MARMLS,A11693223,N,Y,25.883056,-80.121818,https://www.redfin.com/FL/Surfside/9349-Collin...,None,None,None,None,None,None,None,None,2,blue
2,PAST SALE,2025-05-21,Condo/Co-op,300 Biscayne Blvd Way Unit 4802 Duplex,Miami,FL,33131.0,7750000.0,5.0,6.0,Aston Martin Residences,4434.0,NaN,2024.0,NaN,1748.0,7019.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/300-Biscayne-B...,MARMLS,A11691203,N,Y,25.770777,-80.187521,https://www.redfin.com/FL/Miami/300-Biscayne-B...,Silvia Coltrane,The Corcoran Group,None,None,Ivan Chorney,"Compass Florida, LLC.",None,None,3,blue
3,PAST SALE,2025-05-23,Condo/Co-op,300 Biscayne Blvd Way #601,Miami,FL,33131.0,3600000.0,4.0,4.5,ASTON MARTIN CONDO,3360.0,NaN,2024.0,NaN,1071.0,5720.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/300-Biscayne-B...,MARMLS,A11728779,N,Y,25.770777,-80.187521,https://www.redfin.com/FL/Miami/300-Biscayne-B...,Marko Gojanovic,One Sotheby's International Re,Reid Heidenry,One Sotheby's International Re,Maria Sendrowski,Xcellence Realty,None,None,4,blue
4,PAST SALE,2025-05-21,Condo/Co-op,2627 S Bayshore Dr #1201,Miami,FL,33133.0,3600000.0,3.0,3.0,GROVENOR HOUSE,2333.0,NaN,2005.0,NaN,1543.0,3606.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,MARMLS,A11748546,N,Y,25.732272,-80.234921,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,Maria Cerna,EXP Realty LLC,None,None,Ana Sanchez,One Sotheby's International Realty,Yedri Villafane,One Sotheby's International Realty,5,blue
5,PAST SALE,2025-05-20,Condo/Co-op,10101 Collins Ave Unit 15A,Bal Harbour,FL,33154.0,3450000.0,3.0,4.5,THE PALACE AT BAL HARBOUR,3110.0,NaN,1994.0,NaN,1109.0,4755.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10101-Co...,MARMLS,A11799132,N,Y,25.892679,-80.122902,https://www.redfin.com/FL/Bal-Harbour/10101-Co...,Raquel Szyller,"Aventura Real Estate, Inc.",None,None,Nicole Bouhadana,Beachfront Realty Inc,None,None,6,blue
6,PAST SALE,2025-05-23,Condo/Co-op,1643 Brickell Ave #1201,Miami,FL,33129.0,2900000.0,3.0,4.5,SANTA MARIA CONDO,3170.0,NaN,1997.0,NaN,915.0,3626.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1643-Brickell-...,MARMLS,A11659208,N,Y,25.755513,-80.195018,https://www.redfin.com/FL/Miami/1643-Brickell-...,Ingrid Prieto de Ducournau,New Miami Realty,None,None,Ingrid Prieto de Ducournau,New Miami Realty,None,None,7,blue
7,PAST SALE,2025-05-21,Condo/Co-op,488 NE 18th St #4312,Miami,FL,33132.0,2350000.0,3.0,4.0,ARIA ON THE BAY CONDO,2018.0,NaN,2018.0,NaN,1165.0,2239.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/488-NE-18th-St...,MARMLS,A11736645,N,Y,25.793659,-80.186979,https://www.redfin.com/FL/Miami/488-NE-18th-St...,Alessandro Saponaro,AFS Living,None,None,Goritza Draskovich,Fortune International Realty,None,None,8,blue
8,PAST SALE,2025-05-23,Condo/Co-op,4775 Collins Ave #1602,Miami Beach,FL,33140.0,2175000.0,2.0,2.0,GREEN DIAMOND CONDO,1530.0,NaN,2000.0,NaN,1422.0,2062.0,Sold,NaN,NaN,https://www.

In [145]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent {county} County Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [146]:
m.save('index.html')

## Data snagger

In [147]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [148]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-05-20,Condo/Co-op,1425 Brickell Ave Ph 2abcd,Miami,FL,33131.0,14800000.0,4.0,4.0,Four Seasons Residences,7438.0,NaN,2003.0,NaN,1990.0,9800.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1425-Brickell-...,MARMLS,A11702498,N,Y,25.759088,-80.191817,https://www.redfin.com/FL/Miami/1425-Brickell-...,Gabriela Dajer,One Sotheby's International Realty,None,None,Alejandra Anzola,Fortune International Realty,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [149]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MiamiDade_County_condo_sales_week_ending_05272025 


# CREATE TEMPLATE 

In [150]:
muni_set = set(df_top_ten['CITY'])

In [151]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [152]:
df_top_ten.reset_index(inplace=True,drop=True)

In [153]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [154]:
top_sale

'Four Seasons Residences at 1425 Brickell Ave Ph 2abcd in Miami'

In [155]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [156]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [157]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-05-20,Condo/Co-op,1425 Brickell Ave Ph 2abcd,Miami,FL,33131.0,14800000.0,4.0,4.0,Four Seasons Residences,7438.0,NaN,2003.0,NaN,1990.0,9800.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1425-Brickell-...,MARMLS,A11702498,N,Y,25.759088,-80.191817,https://www.redfin.com/FL/Miami/1425-Brickell-...,Gabriela Dajer,One Sotheby's International Realty,None,None,Alejandra Anzola,Fortune International Realty,None,None,1,orange,Four Seasons Residences at 1425 Brickell Ave P...
1,PAST SALE,2025-05-22,Condo/Co-op,9349 Collins Ave #1004,Surfside,FL,33154.0,9000000.0,3.0,4.5,ALTOS DEL MAR NO. 5,3334.0,NaN,2016.0,NaN,2699.0,9757.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9349-Collin...,MARMLS,A11693223,N,Y,25.883056,-80.121818,https://www.redfin.com/FL/Surfside/9349-Collin...,None,None,None,None,None,None,None,None,2,blue,ALTOS DEL MAR NO. 5 at 9349 Collins Ave #1004 ...
2,PAST SALE,2025-05-21,Condo/Co-op,300 Biscayne Blvd Way Unit 4802 Duplex,Miami,FL,33131.0,7750000.0,5.0,6.0,Aston Martin Residences,4434.0,NaN,2024.0,NaN,1748.0,7019.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/300-Biscayne-B...,MARMLS,A11691203,N,Y,25.770777,-80.187521,https://www.redfin.com/FL/Miami/300-Biscayne-B...,Silvia Coltrane,The Corcoran Group,None,None,Ivan Chorney,"Compass Florida, LLC.",None,None,3,blue,Aston Martin Residences at 300 Biscayne Blvd W...
3,PAST SALE,2025-05-23,Condo/Co-op,300 Biscayne Blvd Way #601,Miami,FL,33131.0,3600000.0,4.0,4.5,ASTON MARTIN CONDO,3360.0,NaN,2024.0,NaN,1071.0,5720.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/300-Biscayne-B...,MARMLS,A11728779,N,Y,25.770777,-80.187521,https://www.redfin.com/FL/Miami/300-Biscayne-B...,Marko Gojanovic,One Sotheby's International Re,Reid Heidenry,One Sotheby's International Re,Maria Sendrowski,Xcellence Realty,None,None,4,blue,ASTON MARTIN CONDO at 300 Biscayne Blvd Way #6...
4,PAST SALE,2025-05-21,Condo/Co-op,2627 S Bayshore Dr #1201,Miami,FL,33133.0,3600000.0,3.0,3.0,GROVENOR HOUSE,2333.0,NaN,2005.0,NaN,1543.0,3606.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,MARMLS,A11748546,N,Y,25.732272,-80.234921,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,Maria Cerna,EXP Realty LLC,None,None,Ana Sanchez,One Sotheby's International Realty,Yedri Villafane,One Sotheby's International Realty,5,blue,GROVENOR HOUSE at 2627 S Bayshore Dr #1201 in ...
5,PAST SALE,2025-05-20,Condo/Co-op,10101 Collins Ave Unit 15A,Bal Harbour,FL,33154.0,3450000.0,3.0,4.5,THE PALACE AT BAL HARBOUR,3110.0,NaN,1994.0,NaN,1109.0,4755.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10101-Co...,MARMLS,A11799132,N,Y,25.892679,-80.122902,https://www.redfin.com/FL/Bal-Harbour/10101-Co...,Raquel Szyller,"Aventura Real Estate, Inc.",None,None,Nicole Bouhadana,Beachfront Realty Inc,None,None,6,blue,THE PALACE AT BAL HARBOUR at 10101 Collins Ave...
6,PAST SALE,2025-05-23,Condo/Co-op,1643 Brickell Ave #1201,Miami,FL,33129.0,2900000.0,3.0,4.5,SANTA MARIA CONDO,3170.0,NaN,1997.0,NaN,915.0,3626.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1643-Brickell-...,MARMLS,A11659208,N,Y,25.755513,-80.195018,https://www.redfin.com/FL/Miami/1643-Brickell-...,Ingrid Prieto de Ducournau,New Miami Realty,None,None,Ingrid Prieto de Ducournau,New Miami Realty,None,None,7,blue,SANTA MARIA CONDO at 1643 Brickell Ave #1201 i...
7,PAST SALE,2025-05-21,Condo/Co-op,488 NE 18th St #4312,Miami,FL,33132.0,2350000.0,3.0,4.0,ARIA ON THE BAY CONDO,2018.0,NaN,2018.0,NaN,1165.0,2239.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/488-NE-18th-St...,MARMLS,A11736645,N

In [158]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}, condos, condo market

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: Four Seasons Residences closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,950,000 to $14,800,000
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Miami Beach, Key Biscayne, Surfside, Bal Harbour, Miami, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 86 condo sales totaling $88,050,620 million from May. 17th to May. 25th. The previous week, brokers closed 127 condo sales totaling $123,677,931.

Last week’s units sold for an average of $1,023,844, higher than the $973,842 average pri

In [159]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [160]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [161]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [162]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-05-20,Condo/Co-op,1425 Brickell Ave Ph 2abcd,Miami,FL,33131.0,14800000.0,4.0,4.0,Four Seasons Residences,7438.0,NaN,2003.0,NaN,1990.0,9800.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1425-Brickell-...,MARMLS,A11702498,N,Y,25.759088,-80.191817,https://www.redfin.com/FL/Miami/1425-Brickell-...,Gabriela Dajer,One Sotheby's International Realty,None,None,Alejandra Anzola,Fortune International Realty,None,None,1,orange,Four Seasons Residences at 1425 Brickell Ave P...
1,PAST SALE,2025-05-22,Condo/Co-op,9349 Collins Ave #1004,Surfside,FL,33154.0,9000000.0,3.0,4.5,ALTOS DEL MAR NO. 5,3334.0,NaN,2016.0,NaN,2699.0,9757.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9349-Collin...,MARMLS,A11693223,N,Y,25.883056,-80.121818,https://www.redfin.com/FL/Surfside/9349-Collin...,None,None,None,None,None,None,None,None,2,blue,ALTOS DEL MAR NO. 5 at 9349 Collins Ave #1004 ...
2,PAST SALE,2025-05-21,Condo/Co-op,300 Biscayne Blvd Way Unit 4802 Duplex,Miami,FL,33131.0,7750000.0,5.0,6.0,Aston Martin Residences,4434.0,NaN,2024.0,NaN,1748.0,7019.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/300-Biscayne-B...,MARMLS,A11691203,N,Y,25.770777,-80.187521,https://www.redfin.com/FL/Miami/300-Biscayne-B...,Silvia Coltrane,The Corcoran Group,None,None,Ivan Chorney,"Compass Florida, LLC.",None,None,3,blue,Aston Martin Residences at 300 Biscayne Blvd W...
3,PAST SALE,2025-05-23,Condo/Co-op,300 Biscayne Blvd Way #601,Miami,FL,33131.0,3600000.0,4.0,4.5,ASTON MARTIN CONDO,3360.0,NaN,2024.0,NaN,1071.0,5720.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/300-Biscayne-B...,MARMLS,A11728779,N,Y,25.770777,-80.187521,https://www.redfin.com/FL/Miami/300-Biscayne-B...,Marko Gojanovic,One Sotheby's International Re,Reid Heidenry,One Sotheby's International Re,Maria Sendrowski,Xcellence Realty,None,None,4,blue,ASTON MARTIN CONDO at 300 Biscayne Blvd Way #6...
4,PAST SALE,2025-05-21,Condo/Co-op,2627 S Bayshore Dr #1201,Miami,FL,33133.0,3600000.0,3.0,3.0,GROVENOR HOUSE,2333.0,NaN,2005.0,NaN,1543.0,3606.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,MARMLS,A11748546,N,Y,25.732272,-80.234921,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,Maria Cerna,EXP Realty LLC,None,None,Ana Sanchez,One Sotheby's International Realty,Yedri Villafane,One Sotheby's International Realty,5,blue,GROVENOR HOUSE at 2627 S Bayshore Dr #1201 in ...
5,PAST SALE,2025-05-20,Condo/Co-op,10101 Collins Ave Unit 15A,Bal Harbour,FL,33154.0,3450000.0,3.0,4.5,THE PALACE AT BAL HARBOUR,3110.0,NaN,1994.0,NaN,1109.0,4755.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10101-Co...,MARMLS,A11799132,N,Y,25.892679,-80.122902,https://www.redfin.com/FL/Bal-Harbour/10101-Co...,Raquel Szyller,"Aventura Real Estate, Inc.",None,None,Nicole Bouhadana,Beachfront Realty Inc,None,None,6,blue,THE PALACE AT BAL HARBOUR at 10101 Collins Ave...
6,PAST SALE,2025-05-23,Condo/Co-op,1643 Brickell Ave #1201,Miami,FL,33129.0,2900000.0,3.0,4.5,SANTA MARIA CONDO,3170.0,NaN,1997.0,NaN,915.0,3626.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1643-Brickell-...,MARMLS,A11659208,N,Y,25.755513,-80.195018,https://www.redfin.com/FL/Miami/1643-Brickell-...,Ingrid Prieto de Ducournau,New Miami Realty,None,None,Ingrid Prieto de Ducournau,New Miami Realty,None,None,7,blue,SANTA MARIA CONDO at 1643 Brickell Ave #1201 i...
7,PAST SALE,2025-05-21,Condo/Co-op,488 NE 18th St #4312,Miami,FL,33132.0,2350000.0,3.0,4.0,ARIA ON THE BAY CONDO,2018.0,NaN,2018.0,NaN,1165.0,2239.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/488-NE-18th-St...,MARMLS,A11736645,N

In [163]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: Four Seasons Residences closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,950,000 to $14,800,000
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Miami Beach, Key Biscayne, Surfside, Bal Harbour, Miami, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 86 condo sales totaling $88,050,620 million from May. 17th to May. 25th. The previous week, brokers closed 127 condo sales totaling $123,677,931.

Last week’s units sold for an average of $1,023,844, higher than the $973,842 average pri

In [164]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Miami/1425-Brickell-Ave-33131/unit-2ABCD/home/173271165


In [165]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Surfside/9349-Collins-Ave-33154/unit-1004/home/113411851


In [166]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Key-Biscayne/151-Crandon-Blvd-33149/unit-604/home/42920436


In [167]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Miami-Beach/4775-Collins-Ave-33140/unit-1602/home/43427906


In [168]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Surfside/9349-Collins-Ave-33154/unit-1004/home/113411851


In [169]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Miami/1643-Brickell-Ave-33129/unit-1201/home/42764812


In [170]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-05-20,Condo/Co-op,1425 Brickell Ave Ph 2abcd,Miami,FL,33131.0,14800000.0,4.0,4.0,Four Seasons Residences,7438.0,NaN,2003.0,NaN,1990.0,9800.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1425-Brickell-...,MARMLS,A11702498,N,Y,25.759088,-80.191817,https://www.redfin.com/FL/Miami/1425-Brickell-...,Gabriela Dajer,One Sotheby's International Realty,None,None,Alejandra Anzola,Fortune International Realty,None,None,1,orange,Four Seasons Residences at 1425 Brickell Ave P...
1,PAST SALE,2025-05-22,Condo/Co-op,9349 Collins Ave #1004,Surfside,FL,33154.0,9000000.0,3.0,4.5,ALTOS DEL MAR NO. 5,3334.0,NaN,2016.0,NaN,2699.0,9757.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9349-Collin...,MARMLS,A11693223,N,Y,25.883056,-80.121818,https://www.redfin.com/FL/Surfside/9349-Collin...,None,None,None,None,None,None,None,None,2,blue,ALTOS DEL MAR NO. 5 at 9349 Collins Ave #1004 ...
2,PAST SALE,2025-05-21,Condo/Co-op,300 Biscayne Blvd Way Unit 4802 Duplex,Miami,FL,33131.0,7750000.0,5.0,6.0,Aston Martin Residences,4434.0,NaN,2024.0,NaN,1748.0,7019.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/300-Biscayne-B...,MARMLS,A11691203,N,Y,25.770777,-80.187521,https://www.redfin.com/FL/Miami/300-Biscayne-B...,Silvia Coltrane,The Corcoran Group,None,None,Ivan Chorney,"Compass Florida, LLC.",None,None,3,blue,Aston Martin Residences at 300 Biscayne Blvd W...
3,PAST SALE,2025-05-23,Condo/Co-op,300 Biscayne Blvd Way #601,Miami,FL,33131.0,3600000.0,4.0,4.5,ASTON MARTIN CONDO,3360.0,NaN,2024.0,NaN,1071.0,5720.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/300-Biscayne-B...,MARMLS,A11728779,N,Y,25.770777,-80.187521,https://www.redfin.com/FL/Miami/300-Biscayne-B...,Marko Gojanovic,One Sotheby's International Re,Reid Heidenry,One Sotheby's International Re,Maria Sendrowski,Xcellence Realty,None,None,4,blue,ASTON MARTIN CONDO at 300 Biscayne Blvd Way #6...
4,PAST SALE,2025-05-21,Condo/Co-op,2627 S Bayshore Dr #1201,Miami,FL,33133.0,3600000.0,3.0,3.0,GROVENOR HOUSE,2333.0,NaN,2005.0,NaN,1543.0,3606.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,MARMLS,A11748546,N,Y,25.732272,-80.234921,https://www.redfin.com/FL/Miami/2627-S-Bayshor...,Maria Cerna,EXP Realty LLC,None,None,Ana Sanchez,One Sotheby's International Realty,Yedri Villafane,One Sotheby's International Realty,5,blue,GROVENOR HOUSE at 2627 S Bayshore Dr #1201 in ...
5,PAST SALE,2025-05-20,Condo/Co-op,10101 Collins Ave Unit 15A,Bal Harbour,FL,33154.0,3450000.0,3.0,4.5,THE PALACE AT BAL HARBOUR,3110.0,NaN,1994.0,NaN,1109.0,4755.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10101-Co...,MARMLS,A11799132,N,Y,25.892679,-80.122902,https://www.redfin.com/FL/Bal-Harbour/10101-Co...,Raquel Szyller,"Aventura Real Estate, Inc.",None,None,Nicole Bouhadana,Beachfront Realty Inc,None,None,6,blue,THE PALACE AT BAL HARBOUR at 10101 Collins Ave...
6,PAST SALE,2025-05-23,Condo/Co-op,1643 Brickell Ave #1201,Miami,FL,33129.0,2900000.0,3.0,4.5,SANTA MARIA CONDO,3170.0,NaN,1997.0,NaN,915.0,3626.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1643-Brickell-...,MARMLS,A11659208,N,Y,25.755513,-80.195018,https://www.redfin.com/FL/Miami/1643-Brickell-...,Ingrid Prieto de Ducournau,New Miami Realty,None,None,Ingrid Prieto de Ducournau,New Miami Realty,None,None,7,blue,SANTA MARIA CONDO at 1643 Brickell Ave #1201 i...
7,PAST SALE,2025-05-21,Condo/Co-op,488 NE 18th St #4312,Miami,FL,33132.0,2350000.0,3.0,4.0,ARIA ON THE BAY CONDO,2018.0,NaN,2018.0,NaN,1165.0,2239.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/488-NE-18th-St...,MARMLS,A11736645,N

## Time on Market Calculator

In [98]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 11, 14) ## List (Earlier) date
date2 = datetime(2025, 5, 22) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

189


## Clean CSV for Datawrapper Chart

In [99]:
chart_df = df_top_ten

In [100]:
chart_df = chart_df.fillna(" ")

In [101]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [102]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [103]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [104]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [105]:
chart_df['$/SQUARE FEET'].astype(float)

0    1990.0
1    2699.0
2    1748.0
3    1071.0
4    1543.0
5    1109.0
6     915.0
7    1165.0
8    1422.0
9     970.0
Name: $/SQUARE FEET, dtype: float64

In [106]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [107]:
csv_date_string = today.strftime("%m_%d_%Y")

In [108]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv",index=False)

In [109]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2025-05-20,1425 Brickell Ave Ph 2abcd in Miami,"$14,800,000",4,4.0,"7,438",2003,"$1,990",MARMLS,A11702498,Gabriela Dajer One Sotheby's International Rea...,Alejandra Anzola Fortune International Realty
1,2025-05-22,9349 Collins Ave #1004 in Surfside,"$9,000,000",3,4.5,"3,334",2016,"$2,699",MARMLS,A11693223,,
2,2025-05-21,300 Biscayne Blvd Way Unit 4802 Duplex in Miami,"$7,750,000",5,6.0,"4,434",2024,"$1,748",MARMLS,A11691203,Silvia Coltrane The Corcoran Group,"Ivan Chorney Compass Florida, LLC."
3,2025-05-23,300 Biscayne Blvd Way #601 in Miami,"$3,600,000",4,4.5,"3,360",2024,"$1,071",MARMLS,A11728779,Marko Gojanovic One Sotheby's International Re...,Maria Sendrowski Xcellence Realty
4,2025-05-21,2627 S Bayshore Dr #1201 in Miami,"$3,600,000",3,3.0,"2,333",2005,"$1,543",MARMLS,A11748546,Maria Cerna EXP Realty LLC,Ana Sanchez One Sotheby's International Realty...
5,2025-05-20,10101 Collins Ave Unit 15A in Bal Harbour,"$3,450,000",3,4.5,"3,110",1994,"$1,109",MARMLS,A11799132,"Raquel Szyller Aventura Real Estate, Inc.",Nicole Bouhadana Beachfront Realty Inc
6,2025-05-23,1643 Brickell Ave #1201 in Miami,"$2,900,000",3,4.5,"3,170",1997,$915,MARMLS,A11659208,Ingrid Prieto de Ducournau New Miami Realty,Ingrid Prieto de Ducournau New Miami Realty
7,2025-05-21,488 NE 18th St #4312 in Miami,"$2,350,000",3,4.0,"2,018",2018,"$1,165",MARMLS,A11736645,Alessandro Saponaro AFS Living,Goritza Draskovich Fortune International Realt...
8,2025-05-23,4775 Collins Ave #1602 in Miami Beach,"$2,175,000",2,2.0,"1,530",2000,"$1,422",MARMLS,A11504621,Claudia Whitechurch Coldwell Banker Realty Lau...,Amber Berger The Corcoran Group
9,2025-05-23,151 Crandon Blvd #604 in Key Biscayne,"$1,950,000",3,3.0,"2,010",1980,$970,MARMLS,A11802092,Kim Ellen Haug Fortune Christie's Internationa...,Alicia Malave Cervera Real Estate Inc.
